In [ ]:
# installs and imports
# Clone SwinIR - used to improve picture quality
!git clone https://github.com/JingyunLiang/SwinIR.git
!wget https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth -P experiments/pretrained_models
!pip install big-sleep natsort
!pip install timm
import os, random, shutil, re, lyrics, styles, glob
from tqdm.notebook import trange
from IPython.display import Image, display
from big_sleep import Imagine
from torchvision.utils import save_image
from pathlib import Path
from natsort import natsorted


In [ ]:
# !rm -r -f *  #clear the working directory (when multiple runs are done in one session)
# Create 'sleep' folder for outputs from big sleep

sleep_outputs = 'sleep'

if os.path.isdir(sleep_outputs):
    shutil.rmtree(sleep_outputs)
os.mkdir(sleep_outputs)


# Create 'swin' folder for upsampled outputs    
result_folder = 'swin'

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(result_folder)

In [ ]:
disc = lyrics.albums                                    # list of all albums
print(disc)
album = random.choice(list(disc.values()))              # choose an album
print(album)
song = random.choice(list(album.values()))              # choose a song
lyric = random.choice(song)                             # choose a lyric
print(lyric)
style = random.choice(styles.stylelist) 

# actual prompt
TEXT = f'{lyric} in {style} art style'                  # @param {type: "string"} 
print(TEXT)

SAVE_EVERY = 20                                         # @param {type:"number"}
SAVE_PROGRESS = True                                    # @param {type:"boolean"}
LEARNING_RATE = 5e-2                                    # @param {type:"number"}
ITERATIONS = 500                                        # @param {type:"number"}
SEED = random.randrange(1000000)                        # @param {type: "number"}


# # save prompt info to file
f = open("info.txt", "w")
f.write(lyric + "\n")
f.write(style + "\n")
f.write(str(SEED))
f.close()

model = Imagine(
    text = TEXT,
    save_every = SAVE_EVERY,
    lr = LEARNING_RATE,
    iterations = ITERATIONS,
    save_progress = SAVE_PROGRESS,
    seed = SEED
)

# modify prompt text to get file name created by Imagine
replacements = {" ": "_", ",":"", "|":"--", "-":"_"}
filename = TEXT
for key, value in replacements.items():
    filename = filename.replace(key, value)
for epoch in trange(1, desc = 'epochs'):
    for i in trange(ITERATIONS + 1, desc = 'iteration'): # unclear if the +1 is necessary 
        model.train_step(epoch, i)

        if i== 0 or i % model.save_every != 0:           # only display output after a save
            continue
            
        print(filename, i)                               # keep track of iteration count
        image = Image(f'{filename}.png')                 # get and display the last saved image
        display(image)

In [ ]:
#copy first round of outputs to 'sleep' folder

count = 0

for img in natsorted(os.listdir('.')):                   #sort by counting order, not lexicographical order
    if re.search('[0-9].png', img):                      #last image is saved as filename and filename.24, ignore former
        iterc = '{:0>3}'.format(count)                   #format the count so the new files are automatically sorted correctly
        shutil.copyfile(img, f'{sleep_outputs}/step-{iterc}.png')
        count += 1
        


In [ ]:

# test SwinIR by partioning the image into patches -- idk
test_patch_wise = False

# actually run the upsampling
if test_patch_wise:
  !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq sleep --scale 4 --large_model --tile 640
else:
  !python SwinIR/main_test_swinir.py --task real_sr --model_path experiments/pretrained_models/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth --folder_lq sleep --scale 4 --large_model
shutil.move('results/swinir_real_sr_x4_large', 'swin/SwinIR_large')
for path in natsorted(glob.glob(os.path.join('swin/SwinIR_large', '*.png'))):
  os.rename(path, path.replace('SwinIR.png', 'SwinIR_large.png')) # for compatibility with google colab